In [47]:
import copy
# TODO: investigate death works well and insternal death stats
# TODO: fix die-adaptive lr, show usefull stat instead of logs
# TODO: Collect internal statistics of history of Neurons at their death point (like: values, how fast changed, etc.)
# batch norm analog - when things will go slow with reasonable dataset - likely cifar10, then see if speed improvement
# Separate livenet core and high-level api?

import random
import numpy as np
import torch
import torch.nn as nn
import torch.utils.data
import importlib
import life.lib
import life.lib as lib
import typing
importlib.reload(lib)
LOG = lib.simple_log.LOG
import math
from matplotlib import pyplot as plt
%matplotlib
plt.ion()
lib.utils.set_seed()
print(torch.__version__)
np.set_printoptions(precision=3)
%precision 4



Using matplotlib backend: TkAgg
2.0.1+cu117


'%.4f'

In [2]:
downscale = (8, 8)
test_x, test_y = lib.datasets.to_plain(*lib.datasets.get_cifar10_test(), downscale=downscale)
train_x, train_y = lib.datasets.to_plain(*lib.datasets.get_cifar10_train(), downscale=downscale)


In [51]:
context = lib.livenet.Context()
network = lib.livenet.LiveNet.create_perceptron(train_x.shape[1], 21, 10, context)
batch_iterator = lib.gen_utils.batch_iterator(train_x, train_y, batch_size=1000)
criterion = lib.nets.criterion_n
optimizer = lib.nets.create_optimizer(network)
optimizer.learning_rate = 0.01
trainer = lib.trainer.Trainer(network, batch_iterator, criterion, optimizer, epoch_size=10, adaptive_lr=False)


LiveNet


In [50]:
network.context.alpha_l1 = 0.001
# optimizer.learning_rate = 0.001
# network.context.learning_rate = 0.03
trainer.step(100)


Iˈ0.000 2.780 = 2.444+0.336 params=1244 lr=0.0051 lib/trainer.py:92
Iˈ2.644 2.779 = 2.455+0.323 params=1243 lr=0.0062 lib/trainer.py:92
Iˈ5.155 2.758 = 2.446+0.312 params=1243 lr=0.0076 lib/trainer.py:92
Iˈ7.690 2.743 = 2.436+0.306 params=1241 lr=0.0092 lib/trainer.py:92
Iˈ10.206 2.797 = 2.487+0.310 params=1241 lr=0.0055 lib/trainer.py:92
Iˈ12.731 2.757 = 2.448+0.309 params=1239 lr=0.0033 lib/trainer.py:92
Iˈ15.239 2.742 = 2.435+0.307 params=1237 lr=0.0040 lib/trainer.py:92
Iˈ17.740 2.749 = 2.443+0.306 params=1237 lr=0.0024 lib/trainer.py:92
Iˈ20.216 2.732 = 2.429+0.304 params=1236 lr=0.0014 lib/trainer.py:92
Iˈ22.695 2.726 = 2.425+0.301 params=1234 lr=0.0017 lib/trainer.py:92


In [24]:
pred = network(test_x)
pred_bin = np.argmax(pred.detach().numpy(), axis=1, keepdims=True)
diff = test_y - pred_bin
len(diff[diff != 0]) / len(diff)


0.5305

In [29]:
%matplotlib
plt.ion()

def param_amount(param):
    val = len(param["params"])
    return val

def param_max(param):
    val = np.max(np.abs(param["params"]))
    return val

def param_picker1(param):
    try:
        val0 = param["params"]["S262->D1"].item()
    except KeyError:
        val0 = 0.
    return val0

def get_param_values(history, picker):
    values = []
    for entry in history:
        values.append(picker(entry))
    return values

plt.figure(figsize=(16, 9))
values = get_param_values(trainer.history, param_amount)
plt.grid()
#plt.plot(values)

accum = lib.stat_utils.AccumStat()
accum.add_value(trainer.network.parameters())
accum.plot()


Using matplotlib backend: TkAgg


In [30]:
for name, p in sorted(network.named_parameters(),key=lambda x: len(x[0]) + ord(x[0][0]) / 1000.):
    print(name, f"{p.item():.1f}")


D0 -0.3
D1 -2.4
D2 0.3
D3 1.5
D4 2.3
D5 0.2
D6 -1.8
D7 1.7
D8 -0.1
D9 -1.8
N0 -1.0
N1 0.6
N2 0.1
N3 0.1
N4 0.0
N5 -0.1
N6 0.2
N7 0.7
N8 2.3
N9 0.1
N10 0.2
N11 0.4
N12 1.2
N13 -0.0
N14 -0.1
N15 -0.2
N16 0.1
N17 0.0
N18 0.1
N19 -0.0
N20 -0.4
N0->D0 1.1
N0->D1 2.2
N0->D2 -0.3
N0->D4 0.2
N0->D6 -0.5
N0->D8 0.6
N0->D9 1.6
N1->D0 0.1
N1->D5 0.3
N1->D7 -0.9
N1->D8 1.6
N1->D9 -1.0
N2->D0 -0.5
N2->D2 -0.4
N2->D3 0.6
N2->D4 -1.0
N2->D5 0.7
N2->D7 0.5
N2->D8 -1.0
N2->D9 0.3
N3->D0 -0.4
N3->D1 -0.8
N3->D2 0.3
N3->D3 0.5
N3->D5 0.8
N3->D6 -0.8
N3->D7 0.9
N3->D8 -0.7
N4->D1 0.5
N4->D2 -0.4
N4->D6 -0.3
N4->D7 0.9
N4->D8 0.4
N4->D9 0.9
N5->D0 0.1
N5->D1 0.5
N5->D4 -0.4
N5->D6 0.6
N5->D7 1.0
N5->D8 -0.2
N6->D1 1.5
N6->D2 -0.3
N6->D4 -0.5
N6->D7 -0.5
N6->D8 0.7
N6->D9 1.2
N7->D0 1.0
N7->D2 0.2
N7->D3 -0.3
N7->D5 -0.5
N7->D6 -0.5
N7->D7 0.3
N7->D8 0.7
N7->D9 0.9
N8->D2 0.5
N8->D4 -0.1
N8->D6 1.8
N8->D7 -0.9
N9->D0 0.7
N9->D2 1.3
N9->D3 1.0
N9->D5 1.1
N9->D7 -0.2
N9->D8 -0.8
S0->N0 -0.2
S0->N1 0.1
S5->N9 

In [19]:
network._parameters


OrderedDict([('D0',
              Parameter containing:
              tensor(-0.0146, requires_grad=True)),
             ('D1',
              Parameter containing:
              tensor(0.0146, requires_grad=True)),
             ('N4',
              Parameter containing:
              tensor(-0.0045, requires_grad=True))])

In [151]:
lib.livenet.export_onnx(network, "/home/spometun/model.onnx")


============= Diagnostic Run torch.onnx.export version 2.0.0+cu117 =============
verbose: False, log level: Level.ERROR
======================= 0 NONE 0 NOTE 0 WARNING 0 ERROR ========================
